# 範例 12：簡易 RAG 系統

這個範例展示 RAG 的基本概念：先搜尋文件，再讓 AI 根據文件回答！

## 什麼是 RAG？

**RAG = Retrieval-Augmented Generation（檢索增強生成）**

```
傳統 AI：問題 → AI → 回答（只靠 AI 自己的知識）

RAG AI：問題 → 搜尋文件 → AI + 文件 → 回答（有參考資料！）
```

## 為什麼需要 RAG？
- AI 的知識有截止日期，無法知道最新資訊
- AI 可能「幻想」不存在的資訊
- 企業需要 AI 回答公司內部的專有資料

## 學習目標
- 了解 RAG 的基本架構
- 學會簡單的關鍵字搜尋
- 實作基本的 RAG 系統

## 前置需求
- LM Studio 運行中，Local Server 已啟動
- 安裝 openai 套件：`pip install openai`

## Step 1: 匯入套件

In [ ]:
from openai import OpenAI

# 建立客戶端
client = OpenAI(
    base_url="http://localhost:1234/v1",
    api_key="not-needed"
)

## Step 2: 建立知識庫

這是一個模擬的知識庫。在實際應用中，這可能是：
- 資料庫中的文件
- 檔案系統中的文字檔
- 網頁內容
- 公司內部文件

In [ ]:
# 模擬的知識庫
KNOWLEDGE_BASE = {
    "python": """
    Python 是一種高階程式語言，由 Guido van Rossum 於 1991 年創建。
    Python 的特點：
    - 語法簡潔易讀
    - 支援多種程式設計範式
    - 擁有豐富的第三方套件
    - 廣泛用於網頁開發、資料科學、人工智慧等領域
    """,
    "javascript": """
    JavaScript 是一種腳本語言，主要用於網頁開發。
    JavaScript 的特點：
    - 可在瀏覽器中直接執行
    - 支援事件驅動程式設計
    - 可用於前端和後端（Node.js）開發
    - 是網頁互動功能的核心技術
    """,
    "機器學習": """
    機器學習是人工智慧的一個分支，讓電腦從資料中學習規律。
    機器學習的類型：
    - 監督式學習：使用有標籤的資料訓練
    - 非監督式學習：從無標籤資料中發現模式
    - 強化學習：透過獎勵機制學習最佳策略
    常見應用：圖像辨識、語音辨識、推薦系統等
    """
}

print(f"知識庫中有 {len(KNOWLEDGE_BASE)} 個主題：")
for topic in KNOWLEDGE_BASE.keys():
    print(f"  • {topic}")

## Step 3: 實作簡單的搜尋功能

這是最簡單的關鍵字搜尋。在實際應用中，會使用更先進的方法（如向量搜尋）。

In [ ]:
def simple_search(query):
    """
    簡單的關鍵字搜尋
    在實際應用中，這裡會使用向量搜尋或全文搜尋引擎

    參數：
        query: 搜尋關鍵字

    回傳：
        找到的相關文件內容
    """
    results = []
    query_lower = query.lower()

    for keyword, content in KNOWLEDGE_BASE.items():
        # 檢查關鍵字是否在查詢中，或查詢是否在關鍵字中
        if keyword.lower() in query_lower or query_lower in keyword.lower():
            results.append(content)

    return results

In [ ]:
# 測試搜尋功能
test_queries = ["Python 是什麼", "JavaScript", "什麼是機器學習", "Java 是什麼"]

for query in test_queries:
    results = simple_search(query)
    print(f"查詢：{query}")
    print(f"找到 {len(results)} 筆結果")
    print("-" * 30)

## Step 4: 實作 RAG 聊天函數

RAG 的核心流程：
1. **Retrieval（檢索）**：根據問題搜尋相關文件
2. **Augmentation（增強）**：將找到的文件加入提示詞
3. **Generation（生成）**：讓 AI 根據文件生成回答

In [ ]:
def rag_chat(question):
    """
    RAG 聊天函數
    先搜尋相關文件，再讓 AI 根據文件回答

    參數：
        question: 使用者的問題

    回傳：
        AI 的回答
    """

    # 步驟 1：搜尋相關文件 (Retrieval)
    print("[RAG] 步驟 1：搜尋相關文件...")
    retrieved_docs = simple_search(question)
    print(f"[RAG] 找到 {len(retrieved_docs)} 筆相關文件")

    # 步驟 2：建立提示詞 (Augmentation)
    print("[RAG] 步驟 2：建立增強提示詞...")
    if retrieved_docs:
        context = "\n\n".join(retrieved_docs)
        prompt = f"""請根據以下參考資料回答問題。如果參考資料中沒有相關資訊，請說明你不確定。

參考資料：
{context}

問題：{question}

請用繁體中文回答："""
    else:
        prompt = f"""問題：{question}

注意：我找不到相關的參考資料，請根據你的知識回答，但要說明這是你的一般知識，不是來自特定文件。

請用繁體中文回答："""

    # 步驟 3：呼叫 AI (Generation)
    print("[RAG] 步驟 3：生成回答...")
    response = client.chat.completions.create(
        model="gpt-oss-120b",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

## Step 5: 測試 RAG 系統

In [ ]:
# 測試有資料的問題
question1 = "Python 有什麼特點？"
print(f"問題：{question1}")
print("=" * 50)
answer1 = rag_chat(question1)
print(f"\n回答：{answer1}")

In [ ]:
# 測試沒有資料的問題
question2 = "什麼是區塊鏈？"
print(f"問題：{question2}")
print("=" * 50)
answer2 = rag_chat(question2)
print(f"\n回答：{answer2}")

In [ ]:
# 測試機器學習相關問題
question3 = "機器學習有哪些類型？"
print(f"問題：{question3}")
print("=" * 50)
answer3 = rag_chat(question3)
print(f"\n回答：{answer3}")

## RAG 流程圖解

```
使用者問題
    │
    ▼
┌─────────────┐
│ 1. 搜尋文件 │  ← Retrieval（檢索）
│  (Search)   │
└─────────────┘
    │
    ▼ 找到相關文件
┌─────────────┐
│ 2. 組合提示 │  ← Augmentation（增強）
│   詞+文件   │
└─────────────┘
    │
    ▼ 增強後的提示詞
┌─────────────┐
│ 3. AI 生成  │  ← Generation（生成）
│    回答     │
└─────────────┘
    │
    ▼
  最終回答
```

## Step 6: 擴充知識庫

In [ ]:
# 新增更多知識
KNOWLEDGE_BASE["深度學習"] = """
深度學習是機器學習的一個子領域，使用多層神經網路。
深度學習的特點：
- 可以自動學習特徵
- 需要大量資料訓練
- 計算資源需求高
常見架構：CNN（卷積神經網路）、RNN（循環神經網路）、Transformer
"""

KNOWLEDGE_BASE["API"] = """
API（Application Programming Interface）是應用程式介面。
API 的功能：
- 讓不同程式之間可以互相溝通
- 隱藏複雜的實作細節
- 提供標準化的存取方式
常見類型：REST API、GraphQL、WebSocket
"""

print("知識庫已更新！")
print(f"現在有 {len(KNOWLEDGE_BASE)} 個主題")

In [ ]:
# 測試新增的知識
print("=== 測試深度學習 ===")
print(rag_chat("什麼是深度學習？"))

## 重點回顧

1. **RAG 三步驟**：
   - Retrieval（檢索）：搜尋相關文件
   - Augmentation（增強）：將文件加入提示詞
   - Generation（生成）：AI 根據文件回答

2. **優點**：
   - AI 的回答有依據
   - 可以使用最新或私有資料
   - 減少 AI 幻想

3. **這個範例的限制**：
   - 使用簡單的關鍵字搜尋
   - 知識庫很小
   - 沒有處理長文件

## 下一步

在下一個範例中，我們將學習使用「向量搜尋」來找到更相關的文件！